# Выбор локации для скважины

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Загрузка и подготовка данных

In [109]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Загрузим данные, с которыми нам предстоит работать.

In [110]:
data_one = pd.read_csv('/datasets/geo_data_0.csv')
data_two = pd.read_csv('/datasets/geo_data_1.csv')
data_three = pd.read_csv('/datasets/geo_data_2.csv')

Данные содержат следующие характеристики:
* **id** — уникальный идентификатор скважины;
* **f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);
* **product** — объём запасов в скважине (тыс. баррелей).

In [111]:
data_one.head(10).style.set_caption('Данные геологоразведки в первом регионе').set_table_styles([{
    'selector': 'caption', 'props': [('color', 'blue'), ('font-size', '15px')]
}])

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.22117,105.28
1,2acmU,1.33471,-0.340164,4.36508,73.0378
2,409Wp,1.02273,0.15199,1.41993,85.2656
3,iJLyR,-0.0321716,0.139033,2.97857,168.621
4,Xdl7t,1.98843,0.155413,4.75177,154.037
5,wX4Hy,0.96957,0.489775,-0.735383,64.7415
6,tL6pL,0.645075,0.530656,1.78027,49.0553
7,BYPU6,-0.400648,0.808337,-5.62467,72.9433
8,j9Oui,0.643105,-0.551583,2.37214,113.356
9,OLuZU,2.17338,0.563698,9.44185,127.911


In [112]:
data_two.head(10).style.set_caption('Данные геологоразведки во втором регионе').set_table_styles([{
    'selector': 'caption', 'props': [('color', 'blue'), ('font-size', '15px')]
}])

,id,f0,f1,f2,product
0,kBEdx,-15.0013,-8.276,-0.00587601,3.1791
1,62mP7,14.2721,-3.47508,0.999183,26.9533
2,vyE1P,6.26319,-5.94839,5.00116,134.766
3,KcrkZ,-13.0812,-11.5061,4.99941,137.945
4,AHL4O,12.7022,-8.14743,5.00436,134.766
5,HHckp,-3.32759,-2.20528,3.00365,84.0389
6,h5Ujo,-11.1427,-10.1334,4.00238,110.992
7,muH9x,4.23472,-0.00135394,2.00459,53.9065
8,YiRkx,13.3551,-0.332068,4.99865,134.766
9,jG6Gi,1.06923,-11.0257,4.99784,137.945


In [113]:
data_three.head(10).style.set_caption('Данные геологоразведки в третьем регионе').set_table_styles([{
    'selector': 'caption', 'props': [('color', 'blue'), ('font-size', '15px')]
}])

,id,f0,f1,f2,product
0,fwXo0,-1.14699,0.963328,-0.828965,27.7587
1,WJtFt,0.262778,0.269839,-2.53019,56.0697
2,ovLUW,0.194587,0.289035,-5.58643,62.8719
3,q6cA6,2.23606,-0.55376,0.930038,114.573
4,WPMUX,-0.515993,1.71627,5.89901,149.601
5,LzZXx,-0.758092,0.710691,2.58589,90.2225
6,WBHRv,-0.574891,0.317727,1.77374,45.6415
7,XO8fn,-1.90665,-2.45835,-0.177097,72.4806
8,ybmQ5,1.77629,-0.279356,3.00416,106.617
9,OilcN,-1.21445,-0.439314,5.92251,52.9545


Посмотрим общую информацию о наших датасетах.

In [114]:
datas = [data_one, data_two, data_three]

In [115]:
for data in datas:
    data.info()
    print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage:

Пропусков в данных нет. Проверим данные на дубликаты.

In [116]:
i = 1
for data in datas:
    print('data',i, '-', data.duplicated().sum())
    i += 1

data 1 - 0
data 2 - 0
data 3 - 0


Дубликатов в данных нет. 

Исследуем факторы на мультиколлинерность.

In [117]:
i = 1
for data in datas:  
    print('data', i)
    print(data.corr()) 
    i += 1
    print('\n')

data 1
               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000


data 2
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000


data 3
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000




Данные подготовлены, можно приступать к этапу обучения.

## Обучение и проверка модели

Разделим исходные данные каждого региона на обучающую и валидационную выборки в соотношении 75:25. Обучим модель и сделаем предсказания на валидационной выборке. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [118]:
state = RandomState(12345)
dframes = [data_one, data_two, data_three]

In [119]:
i = 1
for df in dframes:
    print('Region', i)
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = 0.25, random_state = state
    )
    model = LinearRegression().fit(features_train, target_train)
    predictions = model.predict(features_valid) 
    mean_predict = predictions.mean()
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse**0.5
    print('Средний запас предсказанного сырья:', mean_predict)
    print('RMSE модели:', rmse)
    i += 1
    if i <= 3:
        print('\n')

Region 1
Средний запас предсказанного сырья: 92.59256778438038
RMSE модели: 37.5794217150813


Region 2
Средний запас предсказанного сырья: 68.76995145799754
RMSE модели: 0.889736773768064


Region 3
Средний запас предсказанного сырья: 95.087528122523
RMSE модели: 39.958042459521614


### Вывод

* Самый большой объём запасов сырья - в 3-м регионе
* Наименьшая среднеквадратичная ошибка - во 2-м регионе
* Наименьший объём запасов сырья - так же во 2-м регионе

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных.

In [120]:
points = 500
best_points = 200
budget = 10000000000
price_one_barrel = 450
prob_of_loss = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. 

In [121]:
enough_volume = budget / (price_one_barrel * 1000)/ best_points 
print('Достаточный объём сырья для безубыточной разработки новой скважины:', enough_volume, 'тыс. баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111 тыс. баррелей


Сравним полученный объём сырья со средним запасом в каждом регионе.

In [122]:
i = 1
for df in datasets:
    print('Region', i)
    print('Средний запас сырья:', df['product'].mean())
    i += 1

Region 1
Средний запас сырья: 92.50000000000001
Region 2
Средний запас сырья: 68.82500000000002
Region 3
Средний запас сырья: 95.00000000000004


Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [123]:
def oil_profit(target, predictions):
    sorted_predictions = pd.Series(predictions, index = target.index).sort_values(ascending = False)
    appropriate_target = target[sorted_predictions.index][:best_points]
    return appropriate_target.sum() * price_one_barrel * 1000 - budget

### Вывод

Достаточный объём сырья для безубыточной разработки новой скважины равен 111.11 тыс. баррелей. Средние запасы 1, 2 и 3 регионов - 92.5, 68.825 и 95 тыс. баррелей соответственно. Среднего запаса недостаточно, значит, необходимо выбирать скважины, который дают наибольший объём.

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона.

Техника Bootstrap с 1000 выборок.

In [124]:
state = RandomState(555555)
def bootstrap(true_target, pred_target):
    values = []
    loss = 0
    for i in range(1000):
        target_subsample = true_target.sample(n = points, replace = True, random_state = state)
        predictions_subsample = pred_target[target_subsample.index]
        profit = oil_profit(target_subsample, predictions_subsample)
        values.append(profit)
        if profit < 0:
            loss += 1

    values = pd.Series(values)
    lower = values.quantile(q = 0.025)
    upper = values.quantile(q = 0.975)
    mean = values.mean()
    risk = loss / 1000
    
    print('Средняя прибыль:', mean)
    print('95%-й доверительный интервал: от', lower, 'до', upper, 'рублей')
    print('Риск убытка:', risk*100, '%')

Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [125]:
state = RandomState(12345)
datasets = [data_one, data_two, data_three]

In [126]:
m = 1
for df in datasets:
    print('Region', m)
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = 0.25, random_state = state
    )
    model = LinearRegression().fit(features_train, target_train)
    predictions = model.predict(features_valid) 
    target_valid = pd.Series(target_valid).reset_index(drop=True)  # Вот здесь советуют страховаться и ставить .reset_index(drop=True)
    predictions = pd.Series(predictions, index = target_valid.index)
    bootstrap(target_valid, predictions)
    m += 1
    if m <= 3:
        print('\n')

Region 1
Средняя прибыль: 426272370.7738765
95%-й доверительный интервал: от -126790034.5866479 до 983427683.7188534 рублей
Риск убытка: 6.800000000000001 %


Region 2
Средняя прибыль: 509144419.148401
95%-й доверительный интервал: от 103030263.74466619 до 948591900.5358982 рублей
Риск убытка: 1.2 %


Region 3
Средняя прибыль: 345891112.8435471
95%-й доверительный интервал: от -188658819.97242478 до 875761830.8790745 рублей
Риск убытка: 9.8 %


### Вывод

Наименее рисковый вариант для места, где бурить новую скважину, - второй регион (риск убытков - 1.2%). Кроме того, у этого решиона самая большая средняя прибыль - 509144419 рублей.